##**🧩 Problem Statement**

Customer churn is a critical business problem where customers discontinue a service.
The objective of this project is to predict whether a customer will churn based on their demographic information, account details, and service usage.

This is a binary classification problem, where:

*   `1` → Customer will churn
*   `0` → Customer will not churn

Accurate churn prediction helps businesses take proactive retention actions and reduce revenue loss.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


## Dataset Loading

The dataset is loaded using `pandas`. Basic inspection is performed
to understand structure, data types, and missing values.




In [ ]:
df = pd.read_csv("Telco-Customer-Churn.csv")
df.head()

### Data Overview
Checking dataset shape, data types, and missing values.


In [ ]:
df.info()

In [ ]:
df.shape


In [ ]:
df.describe(include='O')

In [ ]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan)

In [ ]:
df['TotalCharges'] = df['TotalCharges'].astype('float64')

In [ ]:
df.info()

In [ ]:
df = df.dropna()

### Class imbalance

In [ ]:
X = df.drop(['Churn', 'customerID'], axis =1)
y = df['Churn']

In [ ]:
y.value_counts(normalize = True)

Splitting the data into training and test sets.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)

In [ ]:
X.describe()

In [ ]:
X.columns

In [ ]:
X.describe(include='O')

## Feature Engineering

Binary categorical features are encoded using `OrdinalEncoder`.
Multi-class categorical features are encoded using `OneHotEncoder`.
Numerical features are scaled using `StandardScaler`.


In [ ]:
binary_cols = ['gender', 'Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
multi_cat_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport', 'StreamingTV','StreamingMovies', 'Contract','PaymentMethod']
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

In [ ]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
binary_transformer = OrdinalEncoder()
multi_cat_transformer = OneHotEncoder(drop='first', sparse_output=False,handle_unknown='ignore')
num_transformer = StandardScaler()


## Model Pipeline
An end-to-end pipeline is created using `ColumnTransformer` and `Pipeline`
to prevent data leakage.


In [ ]:
preprocessor = ColumnTransformer([
    ('binary', binary_transformer, binary_cols ),
    ('multi_cat', multi_cat_transformer, multi_cat_cols),
    ('num', num_transformer, num_cols)
])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import set_config

In [ ]:
set_config(display = 'diagram')

### Training a Logistic Regression model using the pipeline.


In [ ]:
steps = [
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
]


In [ ]:
pipeline = Pipeline(steps)

In [ ]:
pipeline.fit(X_train, y_train)


In [ ]:
y_pred = pipeline.predict(X_test)

In [ ]:
y_pred

## Model Evaluation
Evaluating model performance using confusion matrix and classification report.


In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, classification_report

In [ ]:
confusion_matrix(y_test, y_pred)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
accuracy_score(y_test, y_pred)

### Training a Random Forest classifier using the pipeline.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
     n_estimators=300,
    max_depth=8,
    min_samples_leaf=20,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

In [ ]:
steps2 = [
    ('preprocessor', preprocessor),
    ('classifier', rf)
]

In [ ]:
pipeline2 = Pipeline(steps2)

In [ ]:
pipeline2.fit(X_train, y_train)

In [ ]:
y_pred_rf = pipeline2.predict(X_test)

## Model Evaluation
Evaluating model performance using confusion matrix and classification report.


In [ ]:
print(classification_report(y_test, y_pred_rf))

In [ ]:
confusion_matrix(y_test, y_pred_rf)


### Saving the trained pipeline for deployment.


In [ ]:
import joblib
joblib.dump(pipeline2, "Churn_pipeline.pkl")

In [ ]:
import joblib
pipeline = joblib.load("Churn_pipeline.pkl")


In [ ]:
X_new = pd.DataFrame({
    "gender": ["Male"],
    "Partner": ["Yes"],
    "Dependents": ["No"],
    "PhoneService": ["Yes"],
    "PaperlessBilling": ["Yes"],
    "tenure": [25],
    "SeniorCitizen": [0],
    "MultipleLines": ["No"],
    "InternetService": ["Fiber optic"],
    "OnlineSecurity": ["No"],
    "OnlineBackup": ["Yes"],
    "DeviceProtection": ["No"],
    "TechSupport": ["No"],
    "StreamingTV": ["Yes"],
    "StreamingMovies": ["Yes"],
    "Contract": ["Month-to-month"],
    "PaymentMethod": ["Electronic check"],
    "MonthlyCharges": [85.5],
    "TotalCharges": [900.0]
})

## Threshold Optimization
Using `predict_proba` to apply a custom decision threshold.


In [ ]:
y_prob = pipeline.predict_proba(X_new)


In [ ]:
y_prob_churn = y_prob[:, 1]
print(y_prob_churn)


## Final Results
The optimized threshold improved recall for churned customers,
making the model more suitable for business use.
